In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [3]:
PATH = 'data/aclImdb'

os.makedirs(f'{PATH}/train/all', exist_ok=True)
os.makedirs(f'{PATH}/test/all', exist_ok=True)
os.makedirs(f'{PATH}/models', exist_ok=True)
os.makedirs(f'{PATH}/tmp', exist_ok=True)

TRN_PATH = 'train/all'
VAL_PATH = 'test/all'

TRN = f'{PATH}/{TRN_PATH}'
VAL = f'{PATH}/{VAL_PATH}'

# !!cp -r {PATH}/train/pos/ {TRN}/
# !!cp -r {PATH}/train/neg/ {TRN}/
# !!cp -r {PATH}/train/unsup/ {TRN}/

# !!cp -r {PATH}/test/pos/ {VAL}/
# !!cp -r {PATH}/test/neg/ {VAL}/

%ls {PATH}

aclImdb_v1.tar.gz  imdbEr.txt  imdb.vocab  README  test/  train/


In [4]:
# each review is stored as an individual text file
trn_files = !ls {TRN}

print(f'Total files in /train/all: {len(trn_files)}')
trn_files[:10]

Total files in /train/all: 25001


['0_3.txt',
 '0_9.txt',
 '10000_4.txt',
 '10000_8.txt',
 '10001_10.txt',
 '10001_4.txt',
 '10002_1.txt',
 '10002_7.txt',
 '10003_1.txt',
 '10003_8.txt']

In [5]:
# an example review
review = !cat {TRN}/{trn_files[6]}
review[0]

'Sorry everyone,,, I know this is supposed to be an "art" film,, but wow, they should have handed out guns at the screening so people could blow their brains out and not watch. Although the scene design and photographic direction was excellent, this story is too painful to watch. The absence of a sound track was brutal. The loooonnnnng shots were too long. How long can you watch two people just sitting there and talking? Especially when the dialogue is two people complaining. I really had a hard time just getting through this film. The performances were excellent, but how much of that dark, sombre, uninspired, stuff can you take? The only thing i liked was Maureen Stapleton and her red dress and dancing scene. Otherwise this was a ripoff of Bergman. And i\'m no fan f his either. I think anyone who says they enjoyed 1 1/2 hours of this is,, well, lying.'

In [6]:
# how many words in the dataset (train)
!find {TRN} -name '*.txt' | xargs cat | wc -w

17486581


In [7]:
# how many words in the dataset (val)
!find {VAL} -name '*.txt' | xargs cat | wc -w

5686719


In [8]:
# tokenize = split each sentence into a list of words
' '.join(spacy_tok(review[0]))

'Sorry everyone , , , I know this is supposed to be an " art " film , , but wow , they should have handed out guns at the screening so people could blow their brains out and not watch . Although the scene design and photographic direction was excellent , this story is too painful to watch . The absence of a sound track was brutal . The loooonnnnng shots were too long . How long can you watch two people just sitting there and talking ? Especially when the dialogue is two people complaining . I really had a hard time just getting through this film . The performances were excellent , but how much of that dark , sombre , uninspired , stuff can you take ? The only thing i liked was Maureen Stapleton and her red dress and dancing scene . Otherwise this was a ripoff of Bergman . And i \'m no fan f his either . I think anyone who says they enjoyed 1 1/2 hours of this is , , well , lying .'

In [10]:
# createa torchtext field = describes how to preprocess a piece of text
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [14]:
# create a ModelData object for language modeling
bs = 32 #64
bptt = 70

In [ ]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)

# min_freq = 10 says, "treat any word that appears less than 10 times as the word <unk>"
md = LanguageModelData(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [ ]:
# after building the ModelData object, TEXT.vocab is set.  because this will be needed again, save it
pickle.dump(TEXT, open(f'{PATH}/models/TEXT.pkl', 'wb'))